In [1]:
import cv2

In [2]:
import mediapipe as mp

In [3]:
import threading

In [4]:
# Cell 3: Define hand raise detection and video capture in a thread

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

hands = mp_hands.Hands(
    max_num_hands=10,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

def is_hand_raised(landmarks, frame_height):
    # Wrist landmark (index 0)
    wrist_y = landmarks[0].y * frame_height
    # Hand considered raised if wrist above half the frame height
    return wrist_y < frame_height / 2

def run_hand_raise_detector():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        frame_height, frame_width, _ = frame.shape

        raised_hands_count = 0

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                if is_hand_raised(hand_landmarks.landmark, frame_height):
                    raised_hands_count += 1
                    label = "Hand Raised"
                    color = (0, 255, 0)
                else:
                    label = "Hand Down"
                    color = (0, 0, 255)

                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=3),
                    mp_drawing.DrawingSpec(color=color, thickness=2)
                )

                x_coords = [lm.x for lm in hand_landmarks.landmark]
                y_coords = [lm.y for lm in hand_landmarks.landmark]
                xmin = int(min(x_coords) * frame_width) - 10
                xmax = int(max(x_coords) * frame_width) + 10
                ymin = int(min(y_coords) * frame_height) - 10
                ymax = int(max(y_coords) * frame_height) + 10

                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
                cv2.putText(frame, label, (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.putText(frame, f'Raised Hands: {raised_hands_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        cv2.imshow("Classroom Engagement Tracker", frame)

        key = cv2.waitKey(1)
        if key == 27 or key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [5]:
# Cell 4: Run the hand raise detector function in a thread to avoid blocking notebook

thread = threading.Thread(target=run_hand_raise_detector)
thread.start()
